In [16]:
import geopandas as gpd
import pandas as pd
import os

data_dir = '../data'

# Load the data
tree_dataset = gpd.read_file(data_dir + '/raw_data/geodata_stadt_Zuerich/trees/data/data.gpkg')
flats_duration = gpd.read_file(data_dir + '/derived_data/flats_duration.gpkg')




In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import MultiPolygon
from rasterstats import zonal_stats

def suitability_analysis(
    buffer_dist_vbz: float,
    buffer_trees: float,
    max_slope: float,
    parking_lots_path: str,
    slope_path: str,
    tree_locations_path: str,
    vbz_line_paths: list[str],
    vbz_point_paths: list[str],
    output_path: str
):
    # Read input data
    parking_lots = gpd.read_file(parking_lots_path)
    trees = gpd.read_file(tree_locations_path)
    
    # Step 1: Filter parking types
    parking_filtered = parking_lots[
        ~parking_lots['parking'].isin(['underground', 'multi-storey'])
    ].copy()

    # Step 2: Process VBZ features
    # Merge and buffer line features
    vbz_lines = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_line_paths]),
        crs=parking_lots.crs
    )
    vbz_line_buffers = vbz_lines.buffer(buffer_dist_vbz)

    # Merge and buffer point features
    vbz_points = gpd.GeoDataFrame(
        pd.concat([gpd.read_file(p) for p in vbz_point_paths]),
        crs=parking_lots.crs
    )
    vbz_point_buffers = vbz_points.buffer(buffer_dist_vbz)

    # Step 3: Buffer trees
    tree_buffers = trees.buffer(buffer_trees)

    # Step 4: Combine all buffers
    all_buffers = gpd.GeoSeries(
        list(vbz_line_buffers) + list(vbz_point_buffers) + list(tree_buffers),
        crs=parking_lots.crs
    ).unary_union

    # Step 5: Zonal statistics for slope
    with rasterio.open(slope_path) as src:
        slope_stats = zonal_stats(
            parking_filtered,
            slope_path,
            stats=['mean', 'max'],
            nodata=src.nodata
        )

    # Add slope stats to parking lots
    parking_filtered['slope_mean'] = [s['mean'] for s in slope_stats]
    parking_filtered['slope_max'] = [s['max'] for s in slope_stats]

    # Step 6: Filter by slope
    slope_filtered = parking_filtered[parking_filtered['slope_mean'] <= max_slope]

    # Step 7: Spatial difference with buffers
    final_areas = slope_filtered.geometry.difference(all_buffers)

    # Step 8: Calculate areas and filter
    result = gpd.GeoDataFrame(geometry=final_areas, crs=parking_lots.crs)
    result['area'] = result.geometry.area
    suitable = result[result['area'] >= 16]

    # Save output
    suitable.to_file(output_path, driver='GeoJSON')
    return suitable

# Example usage:
suitability_analysis(
    buffer_dist_vbz=2,
    buffer_trees=5,  # Example tree buffer distance
    max_slope=10,    # Example max slope
    parking_lots_path='parking.gpkg',
    slope_path='slope.tif',
    tree_locations_path='trees.gpkg',
    vbz_line_paths=['vbz_lines1.gpkg', 'vbz_lines2.gpkg'],
    vbz_point_paths=['vbz_points.gpkg'],
    output_path='suitable_parking.geojson'
)


,objid,baumnamedeu,geometry
0,107812,Gemeine Hain- oder Weissbuche,"POLYGON ((2680375.510 1245725.300, 2680375.510..."
1,181771,Blut-Buche (var. purpurea),"POLYGON ((2680382.210 1245656.810, 2680382.210..."
2,217252,"Eibe, Gewöhnliche Eibe","POLYGON ((2680345.615 1245703.179, 2680345.615..."
3,219250,"Eibe, Gewöhnliche Eibe","POLYGON ((2680416.899 1245632.909, 2680416.899..."
4,219730,Rotbuche,"POLYGON ((2680391.689 1245665.660, 2680391.689..."
...,...,...,...
78315,120202,"Douglasie, Douglasfichte","POLYGON ((2682314.430 1251040.900, 2682314.430..."
78316,92365,"Berg-Ahorn, Wald-Ahorn","POLYGON ((2682285.373 1251002.362, 2682285.373..."
78317,96091,Schleppen-Fichte,"POLYGON ((2682380.302 1251026.161, 2682380.302..."
78318,94291,Gemeine Hain- oder Weissbuche,"POLYGON ((2682279.723 1251022.762, 2682279.723..."
